### Installation

In [ ]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow

### Unsloth

Load up `Qwen 2.5 3B Instruct`, and set parameters

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-10 21:08:02 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.9: Fast Qwen2 patching. Transformers: 4.48.3. vLLM: 0.7.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.53%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.3.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### Data Prep

In [ ]:
import re
from datasets import load_dataset, Dataset

SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def format_medical_example(example):
    """
    Transformation function:
      - 'Question': Medical question or case scenario
      - 'Complex_CoT': The model's step-by-step reasoning process
      - 'Response': The result or diagnosis
    Adjust these field names to match those in your dataset.
    """
    reasoning = example.get("Complex_CoT", "Reasoning not provided")
    answer = example.get("Response", "Answer not provided")
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": example["Question"]},
        ],
        "answer": XML_COT_FORMAT.format(reasoning=reasoning, answer=answer)
    }

dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", 'en')
formatted_dataset = dataset["train"].map(format_medical_example)

# Reward functions
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1]) * 0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1) * 0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

README.md:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/74.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25371 [00:00<?, ? examples/s]

Map:   0%|          | 0/25371 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model

In [ ]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = formatted_dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 25,371 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 119,734,272/1,919,856,640 (6.24% trained)


-------------------- Question:
An 11-year-old girl is being evaluated for her sexual development. During the physical examination, she is found to be at Tanner stage 2. What is one of the primary physical signs indicative of Tanner stage 2 in females? 
Answer:
<reasoning>
Alright, let's think this through step by step. An 11-year-old girl is being checked for her sexual development. So, we're looking at Tanner stages to see where she might be. Hmm, okay, Tanner stages are used to describe various phases of physical development in children and teens, especially focusing on things like breast development in girls. Now, for Tanner stage 2, I remember this is kind of early on in puberty.

Oh right, Tanner stage 2 in girls is where we start seeing some initial changes. It's not any major transformation but something more subtle. Okay, the big thing at this stage is breast budding, it's one of those clear tell-tale signs. That's where the breasts start to develop a small mound and there's so

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,-0.000000,-0.207500,0.392967,199.125000,0.000000,-0.207500,0.000000,0.000000,0.000000,0.000000
2,0.000000,-0.126625,0.315913,200.000000,0.000000,-0.126625,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.125000,0.000000,200.000000,0.000013,0.125000,0.000000,0.000000,0.000000,0.000000
4,0.000000,-0.231000,0.380648,200.000000,0.000016,-0.231000,0.000000,0.000000,0.000000,0.000000
5,0.000000,-0.017250,0.203389,184.000000,0.000011,-0.017250,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.125000,0.000000,200.000000,0.000051,0.125000,0.000000,0.000000,0.000000,0.000000
7,0.000000,-0.058000,0.253276,197.750000,0.000018,-0.058000,0.000000,0.000000,0.000000,0.000000
8,0.000000,-0.240500,0.390922,200.000000,0.000014,-0.240500,0.000000,0.000000,0.000000,0.000000
9,0.000000,-0.214250,0.468744,200.000000,0.000018,-0.214250,0.000000,0.000000,0.000000,0.000000
10,0.000000,-0.082125,0.293980,193.750000,0.000017,-0.082125,0.000000,0.000000,0.000000,0.000000


Görüntülenen çıkış son 5000 satıra kısaltıldı.
Now, how do we actually see them in the body? Imaging's the way to go, for sure. But which one? Oh, we have a few options here. There’s CT, MRI, and sometimes even X-rays, though I doubt X-rays are up to the task. CT scans are cool for bone stuff — like, they give a crisp picture of hard structures — but not so great for soft tissues, you know? That makes me think CT might not be the best for schwannomas.

Wait, what about MRI? That could work. Especially if it’s with gadolinium contrast. MRIs are top-notch when it comes to soft tissues because they show these differences really well. Hmm, this could mean MRI can pick out a schwannoma from nearby tissues, which is exactly what we need.

Oh yeah, thinking through this, an MRI with gadolinium seems like it's probably the go-to method. It’s known for being really good at showing soft tissue details, which makes it brilliant for spotting schwannomas in complex areas like where the vestibular n

TrainOutput(global_step=250, training_loss=0.0002141936321621074, metrics={'train_runtime': 6377.4634, 'train_samples_per_second': 0.314, 'train_steps_per_second': 0.039, 'total_flos': 0.0, 'train_loss': 0.0002141936321621074})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template(
    [
        {"role": "user", "content": "A 45-year-old patient presents with chest pain and shortness of breath. What should be the initial diagnostic approach?"}
    ],
    tokenize=False,
    add_generation_prompt=True
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:24<00:00, 24.94s/it, est. speed input: 2.17 toks/s, output: 35.97 toks/s]


"When a 45-year-old patient presents with chest pain and shortness of breath, the initial diagnostic approach should be thorough and comprehensive to rule out various potential causes. Here's a step-by-step initial diagnostic approach:\n\n### Step 1: Detailed History and Physical Examination\n- **History:**\n  - **Chest Pain:**\n    - Location (e.g., central, left or right upper chest)\n    - Quality (e.g., sharp, dull, burning)\n    - Radiation (e.g., to the jaw, left arm)\n    - Onset (sudden, gradual)\n    - Duration (brief, lingering)\n    - Aggravating or alleviating factors (e.g., stress, exertion, eating)\n  - **Shortness of Breath:**\n    - Onset (sudden, gradual)\n    - Duration (brief, lingering)\n    - Activity level (elevated, not related to physical activity)\n    - Aggravating or alleviating factors (e.g., lying down, sitting up)\n  - **Medical History:**\n    - Previous episodes of chest pain\n    - Any history of heart disease, hypertension, diabetes, or high cholestero

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role": "user", "content": "A 45-year-old patient presents with chest pain and shortness of breath. What should be the initial diagnostic approach?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:16<00:00, 16.43s/it, est. speed input: 3.65 toks/s, output: 32.20 toks/s]


"<reasoning>\nThe patient's symptoms of chest pain and shortness of breath could be indicative of several cardiac conditions, including angina, a heart attack (myocardial infarction), or a pulmonary embolism. Therefore, the initial diagnostic approach should aim at differentiating between these possibilities. The key steps in the initial diagnostic approach would include a comprehensive history, a physical examination, and appropriate laboratory and imaging tests. \n\nThe initial history should include details about the onset, duration, severity, and characteristics of the symptoms. This information will guide the choice of diagnostic tests. Commonly, the first step is to obtain an electrocardiogram (ECG) to look for signs of recent heart muscle injury or heart block. \n\nA chest X-ray might be used to check for signs of a pulmonary embolism or other conditions that could cause shortness of breath. Blood tests, such as a complete blood count, liver and kidney function tests, and coagul

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!